In [1]:
import os 
import sys 
sys.path.insert(0, '/home/zhiming/data_extraction/My_FREED')
os.chdir('/home/zhiming/data_extraction/My_FREED')

import numpy as np
import pickle as pkl 

from gym_molecule.envs.env_utils_graph import * 
from gym_molecule.envs.pretrained_models.context_pred.loader import mol_to_graph_data_obj_simple

Using backend: pytorch
/home/zhiming/anaconda3/envs/freed_pt/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [2]:
from gym_molecule.envs.pretrained_models.context_pred.model import GNN, GNN_graphpred 
import torch
from rdkit import Chem 


# import os
# import sys
# module_path = os.path.abspath(os.path.join('...'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

from gym_molecule.envs.molecule_graph import *

In [3]:
device=3
model = GNN_graphpred(5, 300, 1, JK = 'last', drop_ratio = 0.5, graph_pooling = 'mean').to(device)

In [4]:
state = torch.load('gym_molecule/envs/pretrained_models/context_pred/contextpred.pth')
model.gnn.load_state_dict(state)
_ = model.eval()

In [5]:
smi_list, record_list = pkl.load(open(r'./tmp.pkl', 'rb'))

In [6]:
smi, record = smi_list[0], record_list[0]

In [7]:
scaffold = record[0][0]
scaffold = Chem.DeleteSubstructs(scaffold, Chem.MolFromSmiles("*"))
side_chains = [FRAG_VOCAB_MOL_WO_ATT[sc[1]] for _, sc in record]


In [8]:
sizes = [scaffold.GetNumAtoms()] +  [m.GetNumAtoms() for m in side_chains]
graphs = [mol_to_graph_data_obj_simple(scaffold)] + [mol_to_graph_data_obj_simple(m) for m in side_chains]
if device is not None:
    graphs = [g.to(device) for g in graphs]

In [9]:
G_hat = mol_to_graph_data_obj_simple(Chem.MolFromSmiles(smi))
if device is not None:
    G_hat = G_hat.to(device)

In [12]:
G_hat_rep = model.forward_graph_representation(G_hat)

reps = [model.forward_graph_representation(g) for g in graphs]

In [19]:
sizes = torch.tensor(sizes)
if device is not None:
    sizes = sizes.to(device)

tensor([[ 1.0433e-02,  4.3924e-03, -3.4372e-03,  3.3572e-03, -6.7575e-02,
         -5.0604e-03, -1.8692e-03,  4.1106e-01,  6.6599e-05, -7.5092e-06,
         -2.1906e-03,  1.6212e-03,  3.3911e-02, -2.9714e-04,  2.3413e-03,
         -4.6633e-03, -9.8536e-03, -4.9954e-03,  1.4341e-04, -9.2721e-01,
          1.1597e-01, -1.4360e-01, -4.4112e-04,  5.7668e-02,  8.6507e-03,
         -2.3102e-01,  9.8886e-04,  3.3124e-03,  9.5425e-04, -6.4741e-03,
          5.1252e-03,  6.5467e-03, -3.5906e-04,  5.2304e-02, -8.3449e-02,
         -3.0412e-03, -2.6314e-01,  5.5450e-04, -2.0793e-02, -4.6312e-03,
         -1.6025e-03,  2.5223e-05,  2.1891e-01,  1.4829e-03,  7.7551e-03,
          1.6018e-01,  8.1095e-04, -7.2764e-03, -1.6900e-05,  5.7077e-03,
         -1.9073e-03, -1.3306e-02,  1.7296e-03,  3.1017e-03,  1.4971e-02,
         -5.0020e-01,  7.1937e-03, -1.0195e-02,  8.7970e-04,  5.9063e-01,
          3.7137e-03,  7.7840e-04, -8.9766e-02, -8.3407e-03,  4.5344e-03,
          7.6344e-02, -2.9323e-03, -7.

In [26]:
(sizes[:,np.newaxis] * torch.cat(reps)).sum(0, keepdim=True) / sizes.sum() 

tensor([ 1.3332e-02,  4.2216e-03,  3.2643e-01,  3.7550e-03,  3.9277e-02,
        -6.5641e-03, -3.3082e-03, -1.7353e-02,  3.5745e-04, -6.4819e-04,
        -7.7656e-04,  1.9322e-03,  2.4919e-02, -5.9735e-04,  2.7228e-03,
        -3.7424e-03, -1.0799e-02, -3.9753e-03,  1.4567e-04,  4.5255e-03,
        -1.6825e-01,  6.0795e-02,  2.1519e-04,  1.7571e-01, -9.7611e-02,
        -2.3227e-01,  9.9712e-04,  1.3111e-03,  3.0755e-04, -6.8643e-03,
         5.0802e-03, -7.7460e-04, -3.1788e-04,  2.3259e-01, -4.5191e-02,
         3.2605e-03, -1.6713e-01,  5.7592e-04, -7.5044e-03, -4.4487e-03,
         1.1442e-04, -1.9479e-04,  2.6179e-01, -3.5132e-01,  5.1319e-03,
         1.4298e-01,  7.6239e-04, -2.7573e-03, -4.9727e-05,  4.9565e-03,
        -8.7286e-05, -1.5644e-02, -2.2793e-03,  3.1146e-03,  9.4832e-03,
         4.1387e-01,  9.9951e-03, -4.1622e-02,  1.2881e-03,  1.8412e-01,
         3.0311e-03,  8.9894e-04,  1.0066e-02, -8.2931e-03, -9.5274e-03,
        -2.0846e-01, -3.4118e-03, -6.4310e-03,  1.3

In [18]:
torch.cat(reps).shape

torch.Size([5, 300])